In [2]:
import pandas as pd
import numpy as np

df = pd.read_excel('data.xlsx')

In [60]:
df.head(1)

,序号,来源详情,您宝宝的性别,您的宝宝的出生日期：,(1)您有几个孩子,(2)个孩子，这是您的第___个孩子,宝宝的主要抚养人,请问宝宝的家庭类型是,宝宝母亲的年龄（周岁）,宝宝母亲的文化程度:,...,拥有足量的与年龄适宜的玩具,对孩子任何事情都要干涉,给孩子的诉说、表达或申辩的机会,带孩子到商店、公园,赞赏孩子的好奇心，有意识培养孩子兴趣,无端向孩子发脾气,有意识帮助孩子认知事物，扩大词汇,保持孩子活动的房间清洁、整齐,对孩子的需求能够作出及时的回应.1,总分
0,1,0,男,2023-06-20,1,2,保姆阿姨,联合家庭（宝宝、父母、祖父母（或外祖父母）三代一起居住）,23,初中及以下,...,(跳过),(跳过),(跳过),(跳过),(跳过),(跳过),(跳过),(跳过),(跳过),62


In [61]:
df.shape

(52640, 142)

In [62]:
df.columns

Index(['序号', '来源详情', '您宝宝的性别', '您的宝宝的出生日期：', '(1)您有几个孩子', '(2)个孩子，这是您的第___个孩子',
       '宝宝的主要抚养人', '请问宝宝的家庭类型是', '宝宝母亲的年龄（周岁）', '宝宝母亲的文化程度:',
       ...
       '拥有足量的与年龄适宜的玩具', '对孩子任何事情都要干涉', '给孩子的诉说、表达或申辩的机会', '带孩子到商店、公园',
       '赞赏孩子的好奇心，有意识培养孩子兴趣', '无端向孩子发脾气', '有意识帮助孩子认知事物，扩大词汇', '保持孩子活动的房间清洁、整齐',
       '对孩子的需求能够作出及时的回应.1', '总分'],
      dtype='object', length=142)

# 数据预处理

In [63]:
print('总实例数', df.shape[0])
nullValue = df.isnull().sum()
for i in range(len(nullValue)):
    if nullValue[i] != 0 :
        print('缺失特征名:', df.columns[i], '\n缺失:', nullValue[i])

总实例数 52640


缺失值填充，`来源详情` 的 `NAN` 填充为 `0`

In [64]:
df.fillna(0, axis=1, inplace=True)
df.isnull().any()

序号                   False
来源详情                 False
您宝宝的性别               False
您的宝宝的出生日期：           False
(1)您有几个孩子            False
                     ...  
无端向孩子发脾气             False
有意识帮助孩子认知事物，扩大词汇     False
保持孩子活动的房间清洁、整齐       False
对孩子的需求能够作出及时的回应.1    False
总分                   False
Length: 142, dtype: bool

序号和出生日期是无用的，删去这两列

分割目标列，总分

In [65]:
df_drop = df.drop(columns=['序号', '您的宝宝的出生日期：'], axis=1)

data = df_drop.drop('总分', axis=1)
target = df_drop['总分']

feature = data.columns

输出每一个问题的答案种类到文件 `value.txt`

In [66]:
def saveUnique(df, feature, fileName):
    file = open(fileName, 'w+', encoding='utf-8')
    for i in feature:
        print(df[i].unique(), file=file)
    file.close()

def lookUnique(df, feature):
    unq = []
    for i in feature:
        print(df[i].unique())


saveUnique(data, feature, 'value.txt')

In [67]:
decoder = lambda a : dict(zip( a, range(len(a)) ))
dicList = [
    decoder([0, '直接访问']),
    decoder(['男', '女']),
    {'1':1, '2':2, '3':3, '一':1, '二':2, '三':3, '俩':2, '两':2, '两个':2, '一个':1, '4':4,
     '四':4, '5':5, '五':5, '三个':3, '四个':4, '五个':5}, # (1)您有几个孩子
    {'1':1, '2':2, '3':3, '一':1, '二':2, '三':3, '俩':2, '两':2, '两个':2, '一个':1, '4':4,
     '四':4, '5':5, '五':5, '三个':3, '四个':4, '五个':5, '第一':1, '第一个':1, '第二':2, '第二个':2}, # (2)个孩子，这是您的第___个孩子
    {'保姆阿姨':0, '父母':1, '外祖父母或祖父母':2}, # 宝宝的主要抚养人
    decoder(['联合家庭（宝宝、父母、祖父母（或外祖父母）三代一起居住）','核心家庭（宝宝与父母一起居住）','其他（单亲、再婚等）']), # 请问宝宝的家庭类型是
    {}, # 宝宝母亲的年龄（周岁） 
    decoder(['初中及以下','本科','大专','高中及中专','研究生','博士生']), # 宝宝母亲的文化程度:
    {}, # 宝宝父亲的年龄（周岁）
    decoder(['初中及以下','本科','大专','高中及中专','研究生','博士生']), # 宝宝父亲文化的程度:
    decoder(['很好','一般','比较差','比较好','很差']), # 最近一年，您家庭经济情况如何:
    decoder(['县级市', '涉农区', '城区']), # 您宝宝本次体检的社区属于什么区？
    decoder(['0~1岁', '3~6岁', '1~3岁']), # 您的宝宝年龄
]

len(dicList) # 来源详情 + 基本问题 = 1 + 12
for i in range(13):
    data.replace({feature[i]:dicList[i]}, inplace=True)

In [68]:
saveUnique(data, feature, "valueA.txt")

In [69]:
opstr = lambda x: 3 if isinstance(x, str) and x.startswith('其他') else (x if isinstance(x, int) else -1)
data[feature[4]] = data[feature[4]].apply(opstr)

In [70]:
opnum = lambda x: int(x) if isinstance(x, str) and len(x) <= 2 and x.isdigit() else (-1 if isinstance(x, str) else x)
for i in [6, 8]:
    data[feature[i]] = data[feature[i]].apply(opnum)
saveUnique(data, feature, "valueB.txt")

In [71]:
opinvaild = lambda x: -1 if isinstance(x, str) else x
for i in [2, 3]:
    data[feature[i]] = data[feature[i]].apply(opinvaild)
saveUnique(data, feature, "valueC.txt")

In [72]:
mask = ~data.isin([-1]).any(axis=1)
data = data[mask]
data.head(3)

,来源详情,您宝宝的性别,(1)您有几个孩子,(2)个孩子，这是您的第___个孩子,宝宝的主要抚养人,请问宝宝的家庭类型是,宝宝母亲的年龄（周岁）,宝宝母亲的文化程度:,宝宝父亲的年龄（周岁）,宝宝父亲文化的程度:,...,经常强迫孩子多吃一点,拥有足量的与年龄适宜的玩具,对孩子任何事情都要干涉,给孩子的诉说、表达或申辩的机会,带孩子到商店、公园,赞赏孩子的好奇心，有意识培养孩子兴趣,无端向孩子发脾气,有意识帮助孩子认知事物，扩大词汇,保持孩子活动的房间清洁、整齐,对孩子的需求能够作出及时的回应.1
0,0,0,1,2,0,0,23,0,36,3,...,(跳过),(跳过),(跳过),(跳过),(跳过),(跳过),(跳过),(跳过),(跳过),(跳过)
1,0,0,2,2,1,1,30,1,31,3,...,很少,总是,总是,总是,总是,总是,很少,总是,总是,总是
2,0,1,2,2,1,0,30,2,34,2,...,(跳过),(跳过),(跳过),(跳过),(跳过),(跳过),(跳过),(跳过),(跳过),(跳过)


删除异常值(离群点)

In [83]:
def remove_outliers(df):
    # 计算每列的Q1和Q3
    Q1 = df.quantile(0.25)
    Q3 = df.quantile(0.75)
    # 计算IQR
    IQR = Q3 - Q1
    # 定义离群点的条件
    condition = (df < (Q1 - 1.5 * IQR)) | (df > (Q3 + 1.5 * IQR))
    # 删除包含离群点的行
    df_clean = df[~condition.any(axis=1)]
    return df_clean

tmp = remove_outliers(data[feature[[6, 8]]])
data = data.loc[tmp.index]

In [84]:
saveUnique(data, feature, 'result.txt')

In [85]:
data.shape

(48927, 139)

将问题的回答映射为分数

In [86]:
map = {  
    '从不': 1,  
    '很少': 2,  
    '有时': 3,  
    '经常': 4,  
    '总是': 5,
    '(跳过)' : 0,
    '(空)' : 0
}  

for col in data.columns[13:]:
        data[col] = data[col].replace(map)

对负面问题赋权重-1

In [87]:
negative_col = [
    '终日在外忙碌，很少有时间见到孩子',
    '把孩子捆扎得很紧，孩子四肢手足难以活动',
    '不太喜欢孩子，不愿和孩子在一起',
    '为了家庭或孩子的事情争吵、闹别扭',
    '当孩子出现与平常不一样的表现便很着急',
    '总担心孩子出事，把孩子限制得很死',
    '感到带孩子很累、很烦',
    '凭自己心情对孩子忽冷忽热',
    '感到带孩子很烦、很累',
    '用恐怖的意象或故事(如大灰狼)吓唬孩子的自主行为',
    '总担心出事，把孩子限制得很死',
    '惩罚(叱骂、打屁股、罚站等)孩子',
    '不喜欢孩子，不愿和孩子在一起',
    '凭自己心情对孩子忽冷忽热.1',
    '为家庭或孩子的事情而争吵、闹别扭',
    '用恐怖的意象或故事(如大灰狼)吓阻孩子的自主行为',
    '凭自己心情对孩子忽冷忽热.2',
    '亲子出现敌对或冲突',
    '总担心出事，把孩子限制得很死.1',
    '感到带孩子很累、很烦.1',
    '惩罚(叱骂、打屁股、罚站等)孩子.1',
    '为家庭或孩子的事情而争吵',
    '经常强迫孩子多吃一点',
    '对孩子任何事情都要干涉',
    '无端向孩子发脾气'
]

for col in negative_col:
    data[col] *= -1

In [88]:
data.head()

,来源详情,您宝宝的性别,(1)您有几个孩子,(2)个孩子，这是您的第___个孩子,宝宝的主要抚养人,请问宝宝的家庭类型是,宝宝母亲的年龄（周岁）,宝宝母亲的文化程度:,宝宝父亲的年龄（周岁）,宝宝父亲文化的程度:,...,经常强迫孩子多吃一点,拥有足量的与年龄适宜的玩具,对孩子任何事情都要干涉,给孩子的诉说、表达或申辩的机会,带孩子到商店、公园,赞赏孩子的好奇心，有意识培养孩子兴趣,无端向孩子发脾气,有意识帮助孩子认知事物，扩大词汇,保持孩子活动的房间清洁、整齐,对孩子的需求能够作出及时的回应.1
0,0,0,1,2,0,0,23,0,36,3,...,0,0,0,0,0,0,0,0,0,0
1,0,0,2,2,1,1,30,1,31,3,...,-2,5,-5,5,5,5,-2,5,5,5
2,0,1,2,2,1,0,30,2,34,2,...,0,0,0,0,0,0,0,0,0,0
4,0,0,2,2,1,1,33,2,37,3,...,0,0,0,0,0,0,0,0,0,0
5,0,0,2,2,1,0,29,1,30,1,...,0,0,0,0,0,0,0,0,0,0


处理问题列，检查内容是否为整数int，将内容本应该为整数，但含有非法值的样本直接删去

In [89]:
def remove_not_int (df, columns_to_check):
    def is_numeric(s):
        try:
            int(s)
            return True
        except ValueError:
            return False

    mask = df[columns_to_check].applymap(is_numeric).all(axis=1)

    df_filtered = df[mask]

    return df_filtered

In [90]:
columns_to_check = ['(1)您有几个孩子', '(2)个孩子，这是您的第___个孩子', '宝宝母亲的年龄（周岁）', '宝宝父亲的年龄（周岁）'] 
data = remove_not_int(data, columns_to_check)

In [91]:
data['(1)您有几个孩子'] = data['(1)您有几个孩子'].astype(int)
data['(2)个孩子，这是您的第___个孩子'] = data['(2)个孩子，这是您的第___个孩子'].astype(int)

删除不符合常理的数值

In [92]:
data = data[data['(1)您有几个孩子'] <= 10]
data = data[data['(2)个孩子，这是您的第___个孩子'] <= 10]

In [93]:
data.head()

,来源详情,您宝宝的性别,(1)您有几个孩子,(2)个孩子，这是您的第___个孩子,宝宝的主要抚养人,请问宝宝的家庭类型是,宝宝母亲的年龄（周岁）,宝宝母亲的文化程度:,宝宝父亲的年龄（周岁）,宝宝父亲文化的程度:,...,经常强迫孩子多吃一点,拥有足量的与年龄适宜的玩具,对孩子任何事情都要干涉,给孩子的诉说、表达或申辩的机会,带孩子到商店、公园,赞赏孩子的好奇心，有意识培养孩子兴趣,无端向孩子发脾气,有意识帮助孩子认知事物，扩大词汇,保持孩子活动的房间清洁、整齐,对孩子的需求能够作出及时的回应.1
0,0,0,1,2,0,0,23,0,36,3,...,0,0,0,0,0,0,0,0,0,0
1,0,0,2,2,1,1,30,1,31,3,...,-2,5,-5,5,5,5,-2,5,5,5
2,0,1,2,2,1,0,30,2,34,2,...,0,0,0,0,0,0,0,0,0,0
4,0,0,2,2,1,1,33,2,37,3,...,0,0,0,0,0,0,0,0,0,0
5,0,0,2,2,1,0,29,1,30,1,...,0,0,0,0,0,0,0,0,0,0


In [110]:
data = data.reset_index()

In [111]:
data.to_csv('data_to_num.csv', index=False) #用于查看

## 数据探索

### （1）探索不同年龄的养育风险因素

按照年龄将data分为三个部分，分别寻找养育风险因素

In [95]:
data.iloc[:, 13:] = data.iloc[:, 13:].apply(pd.to_numeric, errors='coerce').astype(int)

In [96]:
df1 = data[data['您的宝宝年龄'] == 0]
df2 = data[data['您的宝宝年龄'] == 1]
df3 = data[data['您的宝宝年龄'] == 2]

先对df1分析

In [97]:
column_sums = df1.iloc[:, 13:].sum(axis=0) # 对每一列求和

In [98]:
q15 = column_sums.quantile(0.15)
sorted_sums = column_sums.sort_values()
columns_below_q15 = sorted_sums[sorted_sums.lt(q15)].index

In [99]:
print(sorted_sums) # 对列和进行排列

当孩子出现与平常不一样的表现便很着急              -5041
感到带孩子很累、很烦                      -3039
总担心孩子出事，把孩子限制得很死                -2839
终日在外忙碌，很少有时间见到孩子                -2575
为了家庭或孩子的事情争吵、闹别扭                -2527
                                 ... 
对孩子的需求能够作出及时的回应                  5921
0~1岁儿童家庭养育环境问卷—孩子活动的房间保持清洁、整齐    6020
有意识的和孩子说话                        6081
与孩子有肢体、目光的接触或交流                  6089
孩子由亲生父母亲自看护                      6187
Length: 126, dtype: int64


In [100]:
print("df1后15%的列:")
for column in columns_below_q15:
    print(column)

df1后15%的列:
当孩子出现与平常不一样的表现便很着急
感到带孩子很累、很烦
总担心孩子出事，把孩子限制得很死
终日在外忙碌，很少有时间见到孩子
为了家庭或孩子的事情争吵、闹别扭
凭自己心情对孩子忽冷忽热
把孩子捆扎得很紧，孩子四肢手足难以活动
不太喜欢孩子，不愿和孩子在一起


对df2分析

In [101]:
column_sums = df2.iloc[:, 13:].sum(axis=0) # 对每一列求和

In [102]:
q15 = column_sums.quantile(0.15)
sorted_sums = column_sums.sort_values()
columns_below_q15 = sorted_sums[sorted_sums.lt(q15)].index

In [103]:
print(sorted_sums) # 对列和进行排列

经常强迫孩子多吃一点                 -110554
对孩子任何事情都要干涉                -109682
感到带孩子很累、很烦.1               -108952
用恐怖的意象或故事(如大灰狼)吓阻孩子的自主行为   -104135
为家庭或孩子的事情而争吵               -102901
                             ...  
对孩子的良好行为及时进行肯定、鼓励           196070
鼓励孩子生活自理(吃饭、穿衣、洗漱)          196070
有意识按照公共准则引导和规范孩子的行为.1       197123
感到孩子可爱，愿意和孩子在一起             197574
拥抱、爱抚孩子                     199819
Length: 126, dtype: int64


In [104]:
print("df2后15%的列:")
for column in columns_below_q15:
    print(column)

df2后15%的列:
经常强迫孩子多吃一点
对孩子任何事情都要干涉
感到带孩子很累、很烦.1
用恐怖的意象或故事(如大灰狼)吓阻孩子的自主行为
为家庭或孩子的事情而争吵
惩罚(叱骂、打屁股、罚站等)孩子.1
总担心出事，把孩子限制得很死.1
亲子出现敌对或冲突
凭自己心情对孩子忽冷忽热.2
无端向孩子发脾气


对df3分析

In [105]:
column_sums = df3.iloc[:, 13:].sum(axis=0) # 对每一列求和

In [106]:
q15 = column_sums.quantile(0.15)
sorted_sums = column_sums.sort_values()
columns_below_q15 = sorted_sums[sorted_sums.lt(q15)].index

In [107]:
print(sorted_sums) # 对列和进行排列

感到带孩子很烦、很累                  -7803
总担心出事，把孩子限制得很死              -7170
惩罚(叱骂、打屁股、罚站等)孩子            -6962
用恐怖的意象或故事(如大灰狼)吓唬孩子的自主行为    -6954
为家庭或孩子的事情而争吵、闹别扭            -6640
                            ...  
鼓励孩子用语言表达自己的要求              13620
与孩子经常有肢体、目光、言语的接触或交流        13684
感到孩子可爱，喜欢和孩子在一起             13718
家庭气氛融洽                      13740
亲吻、拥抱、爱抚孩子.1                13957
Length: 126, dtype: int64


In [108]:
print("df3后15%的列:")
for column in columns_below_q15:
    print(column)

df3后15%的列:
感到带孩子很烦、很累
总担心出事，把孩子限制得很死
惩罚(叱骂、打屁股、罚站等)孩子
用恐怖的意象或故事(如大灰狼)吓唬孩子的自主行为
为家庭或孩子的事情而争吵、闹别扭
凭自己心情对孩子忽冷忽热.1
不喜欢孩子，不愿和孩子在一起
